In [2]:
from google.colab import drive
drive.mount('/content/drive/')
import sys
sys.path.append('/content/drive/MyDrive/HW6')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import numpy as np
from tensorflow.keras.datasets import mnist,fashion_mnist
from skimage.util import view_as_windows
from pixelhop import Pixelhop
from skimage.measure import block_reduce
import xgboost as xgb
import warnings, gc
import time
from sklearn.metrics import confusion_matrix, accuracy_score


np.random.seed(1)

# Preprocess
N_Train_Reduced = 10000    # 10000
N_Train_Full = 60000     # 50000
N_Test = 10000            # 10000

BS = 2000 # batch size


def shuffle_data(X, y):
    shuffle_idx = np.random.permutation(y.size)
    X = X[shuffle_idx]
    y = y[shuffle_idx]
    return X, y


def select_balanced_subset(images, labels, use_num_images):
    '''
    select equal number of images from each classes
    '''
    num_total, H, W, C = images.shape
    num_class = np.unique(labels).size
    num_per_class = int(use_num_images / num_class)

    # Shuffle
    images, labels = shuffle_data(images, labels)

    selected_images = np.zeros((use_num_images, H, W, C))
    selected_labels = np.zeros(use_num_images)

    for i in range(num_class):
        selected_images[i * num_per_class:(i + 1) * num_per_class] = images[labels == i][:num_per_class]
        selected_labels[i * num_per_class:(i + 1) * num_per_class] = np.ones((num_per_class)) * i

    # Shuffle again
    selected_images, selected_labels = shuffle_data(selected_images, selected_labels)

    return selected_images, selected_labels

def Shrink(X, shrinkArg):
    #---- max pooling---- (2x2) -to- (1x1)
    pool = shrinkArg['pool']
    # TODO: fill in the rest of max pooling
    if pool:
        N, H, W, C = X.shape
        if H%2!=0 or W%2!=0:
          print("non even height or width")
        pool_out=np.zeros((N,int(H/2),int(W/2),C),dtype=int)
        for n in range(N):
            for c in range(C):
                for h in range(int(H/2)):
                    for w in range(int(W/2)):
                        pool_out[n,h,w,c]=X[n,h*2:h*2+1,w*2:w*2+1,c].max()
    else:
        pool_out=X

    #---- neighborhood construction
    win = shrinkArg['win']
    stride = shrinkArg['stride']
    pad = shrinkArg['pad']
    ch=pool_out.shape[-1]
    # TODO: fill in the rest of neighborhood construction

    if pad>0:
        out=np.pad(pool_out,((0,0), (pad,pad), (pad,pad), (0,0)), 'reflect')
    else:
        out=pool_out
    out = view_as_windows(out, (1, win, win, ch), (1, stride, stride, ch))
    return out.reshape(out.shape[0], out.shape[1], out.shape[2], -1)


# example callback function for how to concate features from different hops
def Concat(X, concatArg):
    return X

def get_feat(X, num_layers=3):
    output = p2.transform_singleHop(X,layer=0)
    if num_layers>1:
        for i in range(num_layers-1):
            output = p2.transform_singleHop(output, layer=i+1)
    return output



warnings.filterwarnings("ignore")
# ---------- Load MNIST data and split ----------
(x_train, y_train), (x_test,y_test) = mnist.load_data()


# -----------Data Preprocessing-----------
x_train = np.asarray(x_train,dtype='float32')[:,:,:,np.newaxis]
x_test = np.asarray(x_test,dtype='float32')[:,:,:,np.newaxis]
y_train = np.asarray(y_train,dtype='int')
y_test = np.asarray(y_test,dtype='int')

# if use only 10000 images train pixelhop
x_train_reduced, y_train_reduced = select_balanced_subset(x_train, y_train, use_num_images=N_Train_Reduced)

x_train /= 255.0
x_test /= 255.0
x_train_reduced /=255.0


In [25]:
# -----------Module 1: set PixelHop parameters-----------
# TODO: fill in this part
SaabArgs = [{'num_AC_kernels':-1, 'needBias':False, 'cw': False},
            {'num_AC_kernels':-1, 'needBias':True, 'cw':True},
            {'num_AC_kernels':-1, 'needBias':True, 'cw':True}]
shrinkArgs = [{'func':Shrink, 'win':5, 'stride': 1,'pad':2,'pool':False},
            {'func': Shrink, 'win':5, 'stride': 1,'pad':0,'pool':True},
            {'func': Shrink, 'win':5, 'stride': 1,'pad':0,'pool':True}]
concatArg = {'func':Concat}

# -----------Module 1: Train PixelHop -----------
# TODO: fill in this part
p2 = Pixelhop(depth=3, TH1=0.005, TH2=0.001,
            SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
p2.fit(x_train_reduced)




=============================================>c/w Saab Train Hop 1
=============================================>c/w Saab Train Hop 2
=============================================>c/w Saab Train Hop 3


In [26]:
p2.save("/content/drive/MyDrive/HW6/Module1_TH1=0.005_TH2=0.001")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
SaabArgs = [{'num_AC_kernels':-1, 'needBias':False, 'cw': False},
            {'num_AC_kernels':-1, 'needBias':True, 'cw':True},
            {'num_AC_kernels':-1, 'needBias':True, 'cw':True}]
shrinkArgs = [{'func':Shrink, 'win':5, 'stride': 1,'pad':2,'pool':False},
            {'func': Shrink, 'win':5, 'stride': 1,'pad':0,'pool':True},
            {'func': Shrink, 'win':5, 'stride': 1,'pad':0,'pool':True}]
concatArg = {'func':Concat}
p2 = Pixelhop(depth=3, TH1=0.005, TH2=0.001,
            SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
p2.load("/content/drive/MyDrive/HW6/Module1_TH1=0.005_TH2=0.001")

In [27]:

# --------- Module 2: get only Hop 3 feature for both training set and testing set -----------
# you can get feature "batch wise" and concatenate them if your memory is restricted
# TODO: fill in this part
x_train_feature=[]
x_test_feature=[]
for i in range(5):
  train_hop3_feats = get_feat(x_train_reduced[2000*i:2000*i+2000]) #get_feat is already set to get 3rd feat
  x_train_feature.append(train_hop3_feats)
  test_hop3_feats = get_feat(x_test[2000*i:2000*i+2000])
  x_test_feature.append(test_hop3_feats)

train_hop3_feats=np.concatenate((x_train_feature[0],x_train_feature[1],x_train_feature[2],x_train_feature[3],x_train_feature[4]),axis=0)
test_hop3_feats=np.concatenate((x_test_feature[0],x_test_feature[1],x_test_feature[2],x_test_feature[3],x_test_feature[4]),axis=0)

#test_hop3_feats = get_feat(x_test)


# --------- Module 2: standardization
STD = np.std(train_hop3_feats, axis=0, keepdims=1)
train_hop3_feats = train_hop3_feats/STD
test_hop3_feats = test_hop3_feats/STD




In [28]:
print(test_hop3_feats)

[[[[ 1.66047649e+01 -1.19745255e+00  1.04089305e-02 ... -3.02700698e-01
     7.98292637e-01  6.43660545e-01]]]


 [[[ 1.77972260e+01 -7.57865787e-01 -1.09365411e-01 ...  1.54463506e+00
     2.08052278e+00 -3.12365198e+00]]]


 [[[ 1.59233589e+01 -1.23661125e+00 -6.38551950e-01 ... -2.14644242e-02
     1.70202985e-01 -2.90413737e-01]]]


 ...


 [[[ 1.85070248e+01  1.89229918e+00  7.48559833e-01 ... -1.05321169e+00
    -2.66358519e+00  2.29826975e+00]]]


 [[[ 1.78256187e+01  1.24818742e+00 -6.65178001e-01 ... -2.14644242e-02
     1.70202985e-01 -2.90413737e-01]]]


 [[[ 1.96710968e+01  1.18099666e+00  2.06043810e-01 ... -3.10354471e-01
    -1.42168093e+00  2.11907005e+00]]]]


In [29]:

train_hop3_feats = np.reshape(train_hop3_feats,(10000,train_hop3_feats.shape[3]))
test_hop3_feats = np.reshape(test_hop3_feats,(10000,test_hop3_feats.shape[3]))

print("train_hop_3_feats shape",train_hop3_feats.shape)
print("test_hop_3_feats shape",test_hop3_feats.shape)


train_hop_3_feats shape (10000, 111)
test_hop_3_feats shape (10000, 111)


In [30]:

#---------- Module 3: Train XGBoost classifier on hop3 feature ---------

tr_acc = []
te_acc = []

clf = xgb.XGBClassifier(n_jobs=-1,
                    objective='multi:softprob',
                    # tree_method='gpu_hist', gpu_id=None,
                    max_depth=6,n_estimators=100,
                    min_child_weight=5,gamma=5,
                    subsample=0.8,learning_rate=0.1,
                    nthread=8,colsample_bytree=1.0)

# train_hop3_feats = np.reshape(train_hop3_feats,(10000,train_hop3_feats.shape[3]))

# test_hop3_feats = np.reshape(test_hop3_feats,(10000,test_hop3_feats.shape[3]))
clf.fit(train_hop3_feats, y_train_reduced)

XGBClassifier(colsample_bytree=1.0, gamma=5, max_depth=6, min_child_weight=5,
              n_jobs=-1, nthread=8, objective='multi:softprob', subsample=0.8)

In [31]:
train_pred = clf.predict(train_hop3_feats)
test_pred = clf.predict(test_hop3_feats)

train_acc = accuracy_score(y_train_reduced, train_pred)
test_acc = accuracy_score(y_test, test_pred)
print("train accuracy",train_acc)
print("test accuracy",test_acc)

train accuracy 0.9701
test accuracy 0.9168


In [19]:
print(test_hop3_feats.shape)
print(test_pred.shape)
print(y_test.shape)
print(test_pred)

(10000, 141)
(10000,)
(10000,)
[5. 5. 5. ... 5. 5. 5.]


In [5]:
# TODO: fill in the rest and report accuracy
# memory burst
# K1=get_feat(x_train,num_layers=0).shape[3]
# K2=get_feat(x_train,num_layers=1).shape[3]
# K3=get_feat(x_train,num_layers=2).shape[3]
# print("total number of parameter", 25*(K1+K2+K3))

K1=len(p2.Energy["Layer0"])
K2=sum([len(i) for i in p2.Energy["Layer1"]])
K3=sum([len(i) for i in p2.Energy["Layer2"]])
print(K1,K2,K3)
print("total number of parameter", 25*(K1+K2+K3))

1 134 111
total number of parameter 6150
